In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data

In [3]:
# filename = '../../__data__/Mary-Anesthesia-20160809-01.mat'
filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/Mary-Anesthesia-20160809-01.mat'
print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

f = h5py.File(filename, 'r')
airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 94.48 seconds)


# VAR

In [ ]:
t = time.localtime()
timestamp = time.strftime('%b-%d-%Y_%H%M', t)
data_path = f"../../../__data__/propofol/VAR"
os.makedirs(data_path, exist_ok=True)

PCA_dim = -1
end_step = T - int(30/dt) # take off 30 seconds

# end_step = 15000
for window in [2.5]:
# for window in [0.15, 0.25, 0.5, 1, 2.5, 5]:
    stride = window
    run_data = {}
    
    # ================
    # RUN PARAMS
    # ================
    run_data['run_params'] = dict(
        window=window,
        stride=stride,
        PCA_dim=PCA_dim,
        end_step=end_step,
        unit_indices={}
    )
    
    # ================
    # AREA BY AREA
    # ================
    
    run_data['results'] = {}
    for area in np.unique(electrode_info['area']):
        areas = [area]
        unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]

        data = lfp[:end_step, unit_indices]
        print(f"window = {window}, area = {area}")
        results = compute_eigs(data, dt, window, stride, return_sigma_norms=True, return_A=True, PCA_dim=PCA_dim, verbose=True)
        results['data_sigma_norms'] = get_data_sigma_norms(data, dt, window, stride)
        del data
        for key in results.keys():
            if len(results[key].shape) > 1:
                results[key] = list(results[key])
        results = pd.DataFrame(results)
        
        run_data['results'][area] = results  
        run_data['run_params']['unit_indices'][area] = unit_indices
    
    # ================
    # ALL AREAS
    # ================
    
    print(f"window = {window}, all areas")
    areas = np.unique(electrode_info['area'])
    unit_indices = np.arange(lfp.shape[1])[pd.Series(electrode_info['area']).isin(areas)]
    data = lfp[:end_step, unit_indices]
    results = compute_eigs(data, dt, window, stride, return_sigma_norms=True, return_A=True, PCA_dim=PCA_dim, verbose=True)
    results['data_sigma_norms'] = get_data_sigma_norms(data, dt, window, stride)
    del data
    for key in results.keys():
        if len(results[key].shape) > 1:
            results[key] = list(results[key])
    results = pd.DataFrame(results)
    
    run_data['results']['all'] = results
    run_data['run_params']['unit_indices']['all'] = unit_indices

    save(run_data, os.path.join(data_path, f"VAR_{os.path.basename(filename)[:-4]}_window_{window}_stride_{stride}_{timestamp}"))

window = 2.5, area = 7b


  0%|          | 0/2822 [00:00<?, ?it/s]

window = 2.5, area = CPB


  0%|          | 0/2822 [00:00<?, ?it/s]

window = 2.5, area = FEF


  0%|          | 0/2822 [00:00<?, ?it/s]

window = 2.5, area = vlPFC


  0%|          | 0/2822 [00:00<?, ?it/s]

window = 2.5, all areas


  0%|          | 0/2822 [00:00<?, ?it/s]